## Example of Batch Processing Using Folder Name with Variable Tags

This example demonstrates how to batch process images using folder name as multiple variable tags. Each folder represents a different set of experimental conditions, and these conditions are extracted from the folder name as variables.Each variable intended to be represented a value in the specified column in the resulting output table. This variables in the name of the folder are delineated by underscores (`_`). The script then used the function to processes paired images within the folder and aggregates the results into master DataFrames, which are then saved as CSV files.

## Explanation

1. **Setup Output Directories**:
    - The script first sets up directories for storing the output data. It creates an `Outputs` folder and a subfolder `Output_images` to store the results and images respectively.

2. **Initialize Data Structures**:
    - Master DataFrames (`master_results` and `master_reports`) and other necessary data structures are initialized to collect the results from each processed folder.

3. **Iterate Over Folders**:
    - The script iterates over each subfolder in the `global_path` directory. Each subfolder represents a different date of experiments.

4. **Process Each Subfolder**:
    - For each subfolder (representing a specific experiment), it extracts variables (such as cells, treatment, concentration) from the folder name. These variables are expected to be separated by underscores (`_`).

5. **Process Paired Images**:
    - The function `in_folder_loop` is called to process the images within each subfolder. It uses the selected analysis function to generate results and reports.

6. **Assign Identifiers**:
    - The extracted variables (cells, treatment, concentration, date) and the name of the selected analysis function are added as columns to the results and reports DataFrames.




In [1]:
import pandas as pd
from pathlib import Path
import natsort
from FLIM_functions import analyze_single_fitted_channel,generate_small_report_fig,analyze_single_fastflim_channel
from helper_functions import in_folder_loop

In [2]:
#global_path=Path(r"F:\SP8_FLIM\Treatments\Aldometanib")
global_path=Path(r"G:\SP8_FLIM\sMol_experiment\Avcute")

In [3]:
selected_analysis_function = analyze_single_fitted_channel

In [19]:
import pandas as pd
from pathlib import Path
import natsort


# Create the folder where all the outputs will be collected 
output_folder_path = global_path / 'Outputs'
output_folder_path.mkdir(exist_ok=True)

outputimage_folder_path = output_folder_path / 'Output_images'
outputimage_folder_path.mkdir(exist_ok=True)

# Initialize master DataFrames and other data structures outside the loop
master_results = pd.DataFrame([])
master_reports = pd.DataFrame([])
report_images = []


# Iterate over all folders in the global_path directory
for day_path in global_path.iterdir():
    if day_path.is_dir() and day_path.stem != 'Outputs':
        
        date = day_path.stem
        print(f"Processing date folder: {date}")
        
        result_per_date = pd.DataFrame()
        
        # level of the folder with analisys, each variable is a 'tag' between _ 
        #For different experiment the columns may be modified depending on the design 
        
        for cell_treatment_path in day_path.iterdir():
            if cell_treatment_path.is_dir(): 
                
                cell_treatment = cell_treatment_path.stem
                
                # Split the folder name to extract cells, age, treatment, and concentration
                try:
                    cells, transfection, treatment  = cell_treatment.split('_')
                
                except ValueError:
                    print(f"Skipping folder with unexpected name format: {cell_treatment}")
                    continue
                
                print(f"Cells: {cells}")
                print(f"Transfection: {transfection}")
                print(f"treatment: {treatment}")
                print(f"  Processing cell_treatment folder: {cell_treatment}")

                result_per_folder,report_per_folder=in_folder_loop(cell_treatment_path,selected_analysis_function)
                
                # identifiction 
                result_per_folder['cells'] = cells
                result_per_folder['transfection'] = transfection
                result_per_folder['treatment'] = treatment

                result_per_folder['Date'] = date
                result_per_folder['selected_analysis'] = selected_analysis_function.__name__

                report_per_folder['cells'] = cells
                report_per_folder['treatment'] = treatment
                report_per_folder['transfection'] = transfection

                report_per_folder['Date'] = date
                report_per_folder['selected_analysis'] =selected_analysis_function.__name__

                # Concatenate the local table to the master DataFrame
                master_results = pd.concat([master_results, result_per_folder])
                master_reports = pd.concat([master_reports, report_per_folder])



# Save the master DataFrame as a CSV file
master_results.to_csv( "master_table.csv")
# Save the full table of stat reports per each image in the tree folders
master_reports.to_csv("master_reports.csv")


Processing date folder: 2024-08-11
Cells: cl1
Transfection: VMAT
treatment: fluox-Acute100
  Processing cell_treatment folder: cl1_VMAT_fluox-Acute100
Cells: cl1
Transfection: VMAT
treatment: imi-Acute100
  Processing cell_treatment folder: cl1_VMAT_imi-Acute100
Cells: cl1
Transfection: VMAT
treatment: pro-Acute100
  Processing cell_treatment folder: cl1_VMAT_pro-Acute100
Cells: cl2
Transfection: VMAT
treatment: fluox-Acute100
  Processing cell_treatment folder: cl2_VMAT_fluox-Acute100
Cells: cl2
Transfection: VMAT
treatment: imi-Acute100
  Processing cell_treatment folder: cl2_VMAT_imi-Acute100
Cells: cl2
Transfection: VMAT
treatment: pro-Acute100
  Processing cell_treatment folder: cl2_VMAT_pro-Acute100
Cells: cl1
Transfection: VMAT
treatment: CQ-Acute100
  Processing cell_treatment folder: cl1_VMAT_CQ-Acute100
Cells: cl2
Transfection: VMAT
treatment: CQ-Acute100
  Processing cell_treatment folder: cl2_VMAT_CQ-Acute100


In [20]:
master_reports

,Mean_fluor_intensity,Mean_tau,Mean_STD,Number_of_objects,Initial_number_of_objects,%_of_object_remained,File Name,cells,treatment,transfection,Date,selected_analysis
0,631.229135,1130.015172,32.062377,142,169,84.023669,1f,cl1,fluox-Acute100,VMAT,2024-08-11,analyze_single_fitted_channel
0,756.338824,963.923432,27.552965,70,89,78.651685,2f,cl1,fluox-Acute100,VMAT,2024-08-11,analyze_single_fitted_channel
0,833.790637,1047.726477,27.944768,58,79,73.417722,3f,cl1,fluox-Acute100,VMAT,2024-08-11,analyze_single_fitted_channel
0,857.039233,1138.102267,27.972363,46,78,58.974359,4f,cl1,fluox-Acute100,VMAT,2024-08-11,analyze_single_fitted_channel
0,499.218623,1263.581844,44.303164,149,177,84.180791,1f,cl1,imi-Acute100,VMAT,2024-08-11,analyze_single_fitted_channel
0,618.867914,1071.130644,34.446017,36,40,90.000000,2f,cl1,imi-Acute100,VMAT,2024-08-11,analyze_single_fitted_channel
0,617.811621,1166.954838,40.254374,103,114,90.350877,3f,cl1,imi-Acute100,VMAT,2024-08-11,analyze_single_fitted_channel
0,659.934710,1100.601396,35.534063,89,111,80.180180,4f,cl1,imi-Acute100,VMAT,2024-08-11,analyze_single_fitted_channel
0,607.673826,1201.986801,38.764720,37,53,69.811321,1f,cl1,pro-Acute100,VMAT,2024-08-11,analyze_single_fitted_channel
0,619.488886,1211.855295,33.517245,124,134,92.537313,2f,cl1,pro-Acute100,VMAT,2024-08-11,analyze_single_fitted_channel


In [21]:
print("Saving master table...")
# Save the master DataFrame as a CSV file
master_results.to_csv(output_folder_path / "master_table2.csv")

print("Saving table of reports...")
# Save the full table of stat reports per each image in the tree folders
master_reports.to_csv(output_folder_path / "master_reports2.csv")

print("Processing complete!")

Saving master table...
Saving table of reports...
Processing complete!
